In [22]:
from openai import OpenAI
import numpy as np
import os
import pandas as pd
import re
import json


### Use of GPT 3.5-turbo-0125
Input: 1.385 tokens
Output: 4096 tokens

Training data up to september 2021

**Re-evaluation with GPT-4 in later stage to see how much better performance is. Main expected benefit is decrease in hallucinations**

MANUALLY ANNOTATED SPEECHES - Available in HLS_man

| COP19        | COP20             | COP21             | COP22         | COP23       | COP24     | COP25     | COP26       | COP27       | COP28       |
|--------------|-------------------|-------------------|---------------|-------------|-----------|-----------|-------------|-------------|-------------|
| japan        | Holy see          | Trinidad & Tobago | Lao           | Netherlands | Malawi    | Serbia    | Russia      | Kazakhstan  | Belgium     |
| namibia      | kenya             | EU                | czechrepublic | Suriname    | Rwanda    | Cambodia  | Grenada     | Canada      | Gambia      |
| sierra leone | micronesia        | Nepal             | israel        | Vanatu      | Indonesia | Uganda    | Philippines | Australia   | Jamaica     |
| usa          | republic of korea | Afghanistan       | belize        | Croatia     | Macedonia | Mauritius | Slovakia    | Serbia      | New Zealand |
| timor leste  | tonga             | Bosnia Herz.      | thailand      | Ireland     | Lesotho   | Malaysia  | Barbados    | South Sudan | Jordan      |
|              | tonga_PSIDS       |                   |               |             |           |           |             |             |             |


In [3]:
with open('HLS_txt_man/cop19_hls_japan.txt', 'r',encoding='utf-8') as file:
    HLS_txt = file.read()

' \nThank you, Mr. President . \n \nOn beha lf of the government of Japan , I would like to express my sinc ere \nappr eciation to His Excellency , Minister Marcin Korolec and his \ngovernment for hosting the COP 19 presidency . \n \nI would also like to expr ess my d eepest condolence to the people of the \nPhilippines and other countries who have been suffering from the r ecent \ntyphoon. \n \nMr. President: \n \nA fair and effective framework with the participation of all countries needs \nto be adopted at COP 21 in 2015 to ensure emissions reduction at the global \nlevel . \n \nIn this regard, Japan firmly supports the establishment of a post-2020 \nframework . \n \nSuch a framework must be based on “nationally -determined” commitment \nand an effective international transparency mechanism. \n \nI will devote myself toward the s uccessful outcome at this COP19 . \n \nMr. President: \n \nThe Great East Japan Earthquake and n uclear Power Plant’ s Accident in \nMarch 2011 caused dras

In [16]:
client = OpenAI()
response = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  seed = 153, #set to maintain consitancy
  temperature = 0.2, # Lower values give more consistent output - range 0-2. Default 1, follow Gilardi 2015 - multiple options
  response_format={"type":"json_object"},
  messages=[
    {
      "role": "system",
      "content": "You will be provided with unstructured speech text, and your task is to label each sentence for relevance in the context of distributive justice of climate policy. Provide a structured overview containing the sentence and the prescribed label. Output JSON."
    },
    {
      "role": "user",
      "content": HLS_txt
    }
  ],
)

#Context of distributive justice is too basic

In [24]:
print(response)

ChatCompletion(id='chatcmpl-9GlO2xgZ2OZfM8i3cgintU6JDx2M5', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n    "sentences": [\n        {\n            "sentence": "A fair and effective framework with the participation of all countries needs to be adopted at COP 21 in 2015 to ensure emissions reduction at the global level .",\n            "label": "relevant"\n        },\n        {\n            "sentence": "In this regard, Japan firmly supports the establishment of a post-2020 framework .",\n            "label": "relevant"\n        },\n        {\n            "sentence": "Such a framework must be based on “nationally -determined” commitment and an effective international transparency mechanism.",\n            "label": "relevant"\n        },\n        {\n            "sentence": "The Great East Japan Earthquake and n uclear Power Plant’ s Accident in March 2011 caused drastic changes in the energy situations .",\n            "label": "i

In [23]:
# Save results
# Specify the file path where you want to save the JSON file
file_path = "GPT_response_file"

# Write the response to a JSON file
with open(file_path, "w") as json_file:
    json.dump(response, json_file)


TypeError: Object of type ChatCompletion is not JSON serializable

TODO find out how to set temperature, max tokens, seed, evaluate system_fingerprint, nr of tokens